# Emission Measure

In [2]:
import os
import sys
import glob
import copy

import numpy as np
from scipy.interpolate import splev,interp1d
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from sunpy.map import Map,GenericMap
import astropy.units as u
from astropy.visualization import ImageNormalize,SqrtStretch,AsinhStretch
from astropy.coordinates import SkyCoord
import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.analysis import DistributedAIACollection,DistributedAIACube
from synthesizAR.analysis.dem import EMCube

sys.path.append('../scripts/')
from dem import HannahKontarModel, make_slope_map_tpeak

import warnings
warnings.filterwarnings('ignore',category=UserWarning,)

%matplotlib inline

In [3]:
cluster = distributed.LocalCluster(n_workers=32,threads_per_worker=2)
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:46542 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 32 Cores: 64 Memory: 270.38 GB


First, read in the model and observational data. For now, we'll just take a time-average. It may be better to time-average in chunks though still not completely sure about that...

In [4]:
read_template = '/storage-home/w/wtb2/data/timelag_synthesis_v2/observational_data/aia/cutouts/aia_lev1.5_201102*_{}_cutout.fits'

In [5]:
aia = InstrumentSDOAIA([0,1]*u.s,None)

In [6]:
cube = DistributedAIACollection(*[DistributedAIACube.from_files(read_template.format(c['name'])) for c in aia.channels])

Next, set up the temperature bins.

In [7]:
temperature_bin_edges = 10.**np.arange(5.5,7.3,0.1) * u.K
temperature_bin_centers = (temperature_bin_edges[1:] + temperature_bin_edges[:-1])/2.

And calculate the instrument responses.

In [8]:
responses = [splev(temperature_bin_centers.value, c['temperature_response_spline']) for c in aia.channels]

Next, we'll time-average the emission in each channel

In [9]:
maps = [cube[c['name']].average(
            chunks=(cube[c['name']].shape[0], cube[c['name']].shape[1]//5, cube[c['name']].shape[2]//5)) 
        for c in aia.channels]

and normalize each map to the exposure time

In [10]:
maps = [Map(m.data/m.meta['exptime'], m.meta) for m in maps]

And finally compute the EM distribution using the method of HK12 in IDL.

In [11]:
hk_model = HannahKontarModel(
    maps,
    temperature_bin_edges,
    responses,
    dem_path='/storage-home/w/wtb2/codes/demreg/idl/'
)

In [12]:
em = hk_model.fit(
    alpha=1,
    increase_alpha=1.1,
    verbose=False,
    n_sample=len(cube[0].maps)
)

Now, compute the emission measure slope in each pixel by fitting over the interval $[10^6,T_{peak}]$, where $T_{peak}$ is the temperature at the peak of the emission measure distribution.

In [25]:
slope_map = make_slope_map_tpeak(
    em,
    Tmin=1e6*u.K,
    safety=10**(0.3),
    rsquared_tolerance=0.9,
)
Map(np.where(slope_map.mask, np.nan, slope_map.data), slope_map.meta).save('../paper/data/observations/em_slope.fits', overwrite=True)

/storage-home/w/wtb2/anaconda3/envs/synthetic-observables/lib/python3.6/site-packages/synthesizAR-0.1.dev856-py3.6.egg/synthesizAR/analysis/dem.py:125: RuntimeWarning: divide by zero encountered in log10
  em_fit = np.log10(data.value.reshape((np.prod(data.shape[:2]),) + data.shape[2:]).T)
/storage-home/w/wtb2/anaconda3/envs/synthetic-observables/lib/python3.6/site-packages/synthesizAR-0.1.dev856-py3.6.egg/synthesizAR/analysis/dem.py:133: RuntimeWarning: divide by zero encountered in true_divide
  rsquared = 1. - rss/rss_flat
/storage-home/w/wtb2/anaconda3/envs/synthetic-observables/lib/python3.6/site-packages/synthesizAR-0.1.dev856-py3.6.egg/synthesizAR/analysis/dem.py:133: RuntimeWarning: invalid value encountered in true_divide
  rsquared = 1. - rss/rss_flat
/storage-home/w/wtb2/anaconda3/envs/synthetic-observables/lib/python3.6/site-packages/synthesizAR-0.1.dev856-py3.6.egg/synthesizAR/analysis/dem.py:134: RuntimeWarning: invalid value encountered in less
  rsquared_mask = rsquared